# Read HTML 

In [ ]:
# Another HTML parser
# !pip3 install lxml

In [ ]:
# !pip install requests

## Parcing HTML

In [ ]:
%%writefile readhtml.py

import requests
import os
from urllib.parse import urljoin
from bs4 import BeautifulSoup

global data
data = dict()

def readHTML(url):
        links = []
        text = ""
        response = requests.get(url)
        if response.status_code != 200:
            print("Error fetching page")
            exit(0)
        else: 
            # BeautifulSoup object represents the document as a nested data structure
            soup = BeautifulSoup(response.text, features="html.parser")           
            # soup = BeautifulSoup(response.text, features="lxml")
        
        # get title
        title = soup.title.get_text() if soup.title else None

        # get description
        meta = soup.find('meta', attrs={'name': 'description'})
        description = meta.get('content') if meta else None     
        
        # remove the scripts and the style elements
        for script in soup(["script", "style"]):
            script.extract() 

        # parse in paragraphs
        selectors = 'h1, h2, h3, p'
        for para in soup.select(selectors):
            text = " ".join((text, str(para.get_text()))) 
        
        # break into lines, remove leading and trailing space on each line
        lines = (line.strip() for line in text.splitlines())

        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)   
        
        links = readLinks(url)                     # internal call
        
        # Store the data
        data[url] = {
                        "title": title, 
                        "description": description,
                        "text": text, 
                        "links": links[url]        
                }
        
        return data
    
    
def readLinks(url):
        external_links = set()
        internal_links = set()
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        # soup = BeautifulSoup(page.content, features="lxml")
        
        # search for specific link
        # for link in soup.findAll('a', attrs={'href': re.compile("^/page/")}):
        
        for line in soup.findAll('a'):
            link = line.get('href')           
            if not link:
                continue
            if link.startswith("javascript:"):
                continue
                
            # Remove anchors
            link = link.split("#")[0]
            
            # Remove parameters
            link = link.split("?")[0]
            
            # Remove trailing forward slash
            link = link.rstrip("/")
            
            if link.startswith('http'):
                external_links.add(link)
            else:
                internal_links.add(link)

        #for link in soup.findAll('a'):
        #    links.append(link.get('href'))
        #return title, links   

        # Full internal links
        full_internal_links = {urljoin(url, internal_link) for internal_link in internal_links}
        
        # All unique external and full internal links
        links_list = list(external_links.union(full_internal_links))        
            
        return links_list   
     
    
# stack is the links_list from above
def readStack(stack):
        print(len(stack))
        visited = []

        while stack:
            # Visit the URL
            current = stack.pop()
            try:
                url = current
                readHTML(url)                     # call
                visited.append(current)
                
            except:
                print(f"Couldn't open {current}")
                continue

        return data
                
                
# Read tables
def readHTMLtab(url):

            response = requests.get(url).text
            # soup = BeautifulSoup(response, 'lxml')
            soup = BeautifulSoup(response, 'html.parser')

            title = soup.title.get_text()
            text = soup.get_text()
            # print(title)
            # print(soup.prettify())  

            # soup.find('table', class_='engineTable') 
            soup.find('table')
            table_body = soup.find('tbody').text

            # remove the starting \n
            table_body = table_body[2:-2]        
            table_list = table_body.split('\n\n\n')

            for index, element in enumerate(table_list):
                table_list[index] = element.split('\n')
                print(table_list)
            return title, table_list

### Test

In [ ]:
import readhtml
from readhtml import readHTML, readLinks, readStack, readHTMLtab

In [ ]:
import importlib 
importlib.reload(readhtml)

In [ ]:
url="https://innotechspace.dk/holodeck/"

In [ ]:
# url = 'https://cphbusiness.dk'

In [ ]:
data = readHTML(url)

In [ ]:
data

In [ ]:
readStack(data[url]['links'])

In [ ]:
data['https://innotechspace.dk/holodeck/'].get('links')

### Test for Links

In [ ]:
links = readLinks(url)
links

### Test for Tabs

In [ ]:
title, table_list = readHTMLtab(url)

In [ ]:
data = pd.DataFrame(data=table_list)
data

Finally, move the whole my dir to '/Users/tdi/opt/anaconda3/lib/python3.9/site-packages'